In [1]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]


In [2]:
data_dir='../data/reber_classification'
tmp_dir='../tmp/reber_classification'

# Fetch the problem
problem = problems.problem("reber_classification")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
problem.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Generating case 0.
INFO:tensorflow:Generated 100000 Examples
INFO:tensorflow:Generating case 0.
INFO:tensorflow:Generated 10000 Examples


In [3]:
# Get the encoders from the problem
encoders = problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None, mode="inputs"):
  """Input str to features dict, ready for inference"""
  inputs = encoders[mode].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers, mode="inputs"):
  """List of ints to str"""
#  integers = list(np.squeeze(integers))
#  if 1 in integers:
#    integers = integers[:integers.index(1)]
  return encoders[mode].decode(np.squeeze(integers))



In [5]:
example = tfe.Iterator(problem.dataset(Modes.EVAL, data_dir)).next()
example_inputs = example["inputs"]
example_label = example["targets"]

print(example)
print("input:", decode(example_inputs, mode="inputs").split())
print("Label: %s" %  encoders["targets"].decode(example_label))

INFO:tensorflow:Reading data files from ../data/reber_classification/reber_classification-dev*
INFO:tensorflow:partition: 0 num_data_files: 1
{'batch_prediction_key': <tf.Tensor: id=98, shape=(1,), dtype=int64, numpy=array([0])>, 'inputs': <tf.Tensor: id=99, shape=(15,), dtype=int64, numpy=array([68, 82, 68, 86, 85, 90, 90, 86, 86, 88, 88, 71, 82, 71,  1])>, 'targets': <tf.Tensor: id=100, shape=(1,), dtype=int64, numpy=array([1])>}
input: ['BPBTSXXTTVVEPE<EOS>']
Label: True
